In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.applications.xception import Xception
import os
from pickle import dump, load



2021-11-19 13:20:39.723146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-19 13:20:39.723192: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
     		if index == integer:
         	    return word
	return None

In [3]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [4]:
def extract_features(filename, model):
        try:
            image = Image.open(filename)

        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

In [5]:
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_1.h5')
xception_model = Xception(include_top=False, pooling="avg")
max_length = 32

2021-11-19 13:20:45.272682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-19 13:20:45.272731: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-19 13:20:45.272767: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pinkPrincess): /proc/driver/nvidia/version does not exist
2021-11-19 13:20:45.273087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def test_image(file):
    photo = extract_features(file, xception_model)
    img = Image.open(file)

    description = generate_desc(model, tokenizer, photo, max_length)
    print(description)

In [7]:
test_image("t1.jpg")

start boy in red shirt is jumping into the water end


In [8]:
test_image("t2.jpg")

start man in red shirt is standing on bench end


In [9]:
test_image("t3.jpg")

start little boy is playing on the swing end


In [10]:
test_image("t4.jpeg")

start man in black shirt is standing on bench end


In [11]:
test_image("t5.png")

start boy in red shirt is jumping into the air end


In [12]:
test_image("t6.png")

start man in black shirt is standing on the street end


In [13]:
test_image("t7.jpg")

start two people are riding on the street end


In [14]:
test_image("t8.jpg")

start man in red shirt is playing soccer end
